In [1]:
from math import pi,cos,sqrt,log,exp

In [31]:
from folium import Map,Marker,Icon
from folium.features import PolyLine
from folium.plugins import HeatMap
m = Map(location=[52.516422, 13.377767], zoom_start=11)

# Knowledge

## Distance to the Spree

The path of the river Spree through the city of Berlin is approximated in a piecewise linear manner.

In [3]:
spree_geo = [(52.529198,13.274099),
             (52.531835,13.29234),
             (52.522116,13.298541),
             (52.520569,13.317349),
             (52.524877,13.322434),
             (52.522788,13.329),
             (52.517056,13.332075),
             (52.522514,13.340743),
             (52.517239,13.356665),
             (52.523063,13.372158),
             (52.519198,13.379453),
             (52.522462,13.392328),
             (52.520921,13.399703),
             (52.515333,13.406054),
             (52.514863,13.416354),
             (52.506034,13.435923),
             (52.496473,13.461587),
             (52.487641,13.483216),
             (52.488739,13.491456),
             (52.464011,13.503386),
             (52.448181,13.572909),
             (52.444107,13.625790)]

In [32]:
PolyLine(spree_geo,popup='Spree river (piecewise linear approximation)').add_to(m)
m

The knowledge of the distance $D_{Spree}$ of the candidate to the Spree translates as follows:

$$
D_{Spree} \sim \mathcal{N}(0,\sigma_{Spree})
$$

with $\sigma_{Spree}= 2.730 / 2\,km$, as the 95 % confidence interval has a radius of 2.730 km, and this radius is (approximately) equal to $2 \times \sigma_{Spree}$.

In [5]:
std_spree = 2.730 / 2
mean_spree = 0.0

## Distance to the Brandenburger Tor

The Brandenburger Tor has the following coordinates:

In [6]:
tor_geo = (52.516288,13.377689)

The knowledge of the distance $D_{Tor}$ of the candidate to the Spree translates as follows:

$$
log(D_{Tor}) \sim \mathcal{N}(\mu,\sigma_{Tor})
$$

with:

$$
\mu = \log \mathbb{E}(D_{Tor}) - \frac{\sigma_{Tor}^2}{2}
$$

and:

$$
\sigma = \sqrt{\frac{2}{3} \times (\log(\mathbb{E}(D_{Tor}))-\log(mode(D_{Tor}))}
$$

In [7]:
mean_tor = 4.7
mode_tor = 3.877
std_tor = sqrt((log(mean_tor)-log(mode_tor)) * 2/3)
mu_tor = log(mean_tor) - std_tor**2 / 2

In [33]:
Marker(location=tor_geo,popup='Brandenburger Tor',icon=Icon(icon='')).add_to(m)
m

## Distance to a satellite path

The satellite follows a straight line joining the two following points:

In [9]:
sat_geo = [(52.590117,13.39915),
           (52.437385,13.553989)]

The knowledge of the distance $D_{Sat}$ of the candidate to this path translates into:

$$
D_{Sat} \sim \mathbb{N}(0,\sigma_{Sat})
$$

with $\sigma_{Sat} = 2.4 / 2 km$, half of the range of the centered 95% confidence interval.

In [10]:
std_sat = 2.4 / 2
mean_sat = 0.0

earth_radius = 6371 # in km

In [34]:
PolyLine(sat_geo,popup="Satellite path").add_to(m)
m

# Utility functions

Spatial coordinates are converted into cartesian coordinates in the tangent space at the Brandenburger Tor,
which is also the origin of the linearized space.

In [12]:
def lng_radius(lat):
    '''
    Radius of the parallel at `lat`
    '''
    return earth_radius * cos(lat * pi/180)
    
def geo_to_cartesian(lat,lng,origin_lat=tor_geo[0],origin_lng=tor_geo[1]):
    '''
    Linearize geographic coordinates to cartesian coordinates
    '''
    x = (lat-origin_lat) * earth_radius * pi/180
    y = (lng-origin_lng) * pi/180 * lng_radius(origin_lat)
    
    return x,y

def cartesian_to_geo(x,y,origin_lat=tor_geo[0],origin_lng=tor_geo[1]):
    lat = origin_lat + x/earth_radius * 180/pi
    lng = origin_lng + y/lng_radius(origin_lat) * 180/pi
    
    return lat,lng
    
def distance(*args,bounded=False):
    '''
    Compute the distance to a point or a line
    '''
    if len(args) == 4:
        return sqrt((args[0]-args[2])**2 + (args[1]-args[3])**2)
    
    if len(args) == 6:
        x,y = args[0:2]
        x_a, y_a, x_b, y_b = args[2:]
        u,v = x_b - x_a, y_b - y_a

        c = ((x-x_a)*u + (y-y_a)*v) / (u**2 + v**2)
        p = (x_a + c*u, y_a + c*v)
        if bounded:
            if c < 0:
                p = (x_a,y_a)
            if c > 1:
                p = (x_b,y_b)
        return distance(x,y,*p)
    
def distance_multi(x,y,vertices):
    lines = zip(vertices[::2],vertices[1::2])
    distances = []
    for l in lines:
        distances.append(distance(x,y,l[0][0],l[0][1],*l[1],bounded=True))
    return min(distances)

# Likelihood

In [13]:
from scipy.stats import norm,lognorm

spree_cart = [geo_to_cartesian(*p) for p in spree_geo]
def logp_spree(x,y):
    d = distance_multi(x,y,spree_cart)
    return norm.logpdf(d,loc=mean_spree,scale=std_spree)

tor_cart = geo_to_cartesian(*tor_geo)
def logp_tor(x,y):
    d = distance(x,y,*tor_cart)
    return lognorm.logpdf(d,std_tor,scale=exp(mu_tor))

sat_cart = [geo_to_cartesian(*p) for p in sat_geo]
def logp_sat(x,y):
    d = distance(x,y,sat_cart[0][0],sat_cart[0][1],*sat_cart[1])
    return norm.logpdf(d,loc=mean_sat,scale=std_sat)

The three sources of information are independent, the likelihood of an observation `x` is the product of
each probability.

In [14]:
def nlikelihood(x):
    return logp_spree(*x) + logp_tor(*x) + logp_sat(*x)

# Optimization

In order to produce a point prediction, some would advocate a maximum-likelihood approach.

In [15]:
from scipy.optimize import minimize
import numpy as np

In [16]:
x0 = np.array((1.0,1.0))
res = minimize(lambda x: -nlikelihood(x),x0,method='nelder-mead',
               options={'xtol': 1e-4})

mlike = cartesian_to_geo(*res.x)

In [35]:
Marker(location=mlike,popup='Maximum likelihood',icon=Icon(icon=None,color='red')).add_to(m)
m

# Sampling

There's more information available by exploring the posterior distribution (assuming a uniform prior over the area of interest, though one could build a prior from public census data).

Here we use a basic Monte Carlo Markov chain.

In [18]:
from numpy.random import uniform,random_sample
from numpy import average

sample_size = 50000
burnin_size = 10000

In [28]:
values_geo,values_cart = [],[]
x_old, y_old = res.x
lat_old, lng_old = cartesian_to_geo(x_old,y_old)
lik_old = 0.0
for ii in range(sample_size):
        x = x_old + uniform(-1,1)
        y = y_old + uniform(-1,1)
        lik = nlikelihood([x,y])
        if random_sample() < exp(lik-lik_old):
            values_cart.append((x,y))
            lat,lng = cartesian_to_geo(x,y)
            values_geo.append((lat,lng))
            x_old,y_old,lat_old,lng_old,lik_old = x,y,lat,lng,lik
        else:
            values_geo.append((lat_old,lng_old))
            values_cart.append((x_old,y_old))

## Optimal stand point for the recruiter

If there is only one recruiter available, then the best location to stand at is given by the posterior average.

In [29]:
post_avg = average(values_cart[burnin_size::50],axis=0)
post_avg_lat,post_avg_lng = cartesian_to_geo(*post_avg)

In [36]:
Marker(location=[post_avg_lat,post_avg_lng], icon=Icon(icon='',color='green'),popup='Posterior average').add_to(m)
m

## Posterior distribution of the location of the candidate

If several recruiters are available, they should be dispatched according to the posterior distribution.
Ideally, one would derive it from public census data.

In [37]:
HeatMap(values_geo[burnin_size::50],radius=20,blur=22).add_to(m)
m